# VK API

(этот конспект частично повторяет материал, написанный Б.Ореховым [вот здесь](https://github.com/elmiram/2016learnpython/blob/master/VK%20API%20%D0%A7%D0%B0%D1%81%D1%82%D1%8C%201.ipynb))

## Введение

Социальные сети - источник большого количества текстов на современном естественном языке, а потому очень полезны компьютерным лингвистам. Но скачивать содержимое страниц в социальной сети бывает не так просто. 

Некоторые ресурсы либо прямо запрещают автоматическое обращение к своему контенту, потому что это для них невыгодно (зарабатывают-то они на рекламе, которую показывают **людям**, а с роботов отдача нулевая) и излишне загружает сервера, либо просто организовывают свой сайт так, чтобы сама по себе загрузка страницы клиентом не давала ничего существенного, а всё нужное подгружалось уже потом с помощью программ на языке JavaScript. Запускать такие программы из питона несколько сложнее, так что всё это создаёт для компьютерного лингвиста дополнительные трудности.

Однако многие крупные ресурсы либо по доброте душевной, либо потому что это даёт и им кое-какую выгоду, встраивают в свою систему API: application programming interface, то есть средство для автоматизированного обращения к приложению (сайту). Через такую систему можно решать иногда довольно широкий спектр задач. Может быть, всё, что таким образом можно сделать, нам не нужно. Но вот получать тексты было бы полезно: в тех же социальных сетях люди пишут, во-первых, много (а для компьютерного лингвиста чем больше данных, тем лучше), во-вторых, на таком варианте языка, который приближен к разговорному (другие способы намайнить себе текстов такого рода гораздо затратнее). Попробуем познакомиться с инструментарием API на примере vk.com


## Как выглядит обращение к VK API

В принципе, VK API, как следует из [документации](https://vk.com/dev/openapi) придуман не для выкачивания текстов, а для создания веб-приложений на сторонних сайтах, которые бы могли взаимодействовать с vk.com. Но это всё равно не мешает нам воспользоваться такой возможностью в своих целях.

Практически всё выглядит следующим образом. На сайте vk.com есть специальные страницы, которые не предназначены для того, чтобы их открывать браузером, они ожидают именно автоматического обращения. Что значит *автоматическое обращение*? Это то самое, что мы проходили, когда учились выкачивать Интернет: программа на питоне (но в теории может быть и не на питоне) посылает серверу запрос, договаривается с ним и получает ответ. Вспомним, что для произвольной страницы это выглядит так:

In [2]:
import requests
response = requests.get('http://ya.ru')
response.text[:200]

'<!DOCTYPE html><html class="i-ua_js_no i-ua_css_standart i-ua_browser_unknown i-ua_browser_desktop i-ua_platform_other" lang="ru"><head xmlns:og="http://ogp.me/ns#"><meta http-equiv="X-UA-Compatible" '

Для взаимодействия с vk.com нам потребуется точно так же отправлять http-запросы с помощью модуля requests, а страницы, к которым мы будем обращаться, описаны в [документации VK API](https://vk.com/dev/openapi).

Страницы, к которым мы будем обращаться, ожидают не просто обращения, а передачи определённых параметров. Это логично: мы же должны сказать системе, что мы точно хотим сделать. Например, мы хотим получить какое-то количество записей со стены определённого пользователя. Тогда нужно сообщить, какой это пользователь и сколько нам нужно записей. 

### Передача параметров

Параметры передаются в http-запросе: 

* мы отправляем запрос на какую-то страницу, например, https://api.vk.com/method/wall.get
* чтобы передать параметры, после адреса мы пишем знак вопроса `?`, и перечисляем параметры как пары ключ-значение, где ключ отделяется от значения с помощью знака равно, разные параметры должны отделяться друг от друга знаком "аперсанд", то есть **&**:

https://api.vk.com/method/wall.get?owner_id=1&count=10

Вот так мы можем отправить запрос к сети ВК, чтобы получить первые 10 записей (`count=10`) со стены (`wall.get`) пользователя с номером 1 (`owner_id=1`).

`wall.get` - это название метода в API, к которому мы обращаемся, чтобы получить записи со стены. Как работает этот метод, рассказано на его странице в документации: https://vk.com/dev/wall.get Среди полезного там есть список обязательных параметров, которые нужно передать методу, чтобы он сработал.

Если мы хотим обратиться к другому методу (например, получить информацию о пользователе, о сообществе, скачать переписку и т.д.), то вместо `wall.get` нужно указать его название, а также проверить в документации, какие параметры ожидает этот метод.



### Запрос и результат

Давайте скачаем две записи со стены Павла Дурова. 

In [3]:
response = requests.get('https://api.vk.com/method/wall.get?owner_id=1&count=2')
response.text

'{"response":[237,{"id":1725537,"from_id":1,"to_id":1,"date":1492437980,"post_type":"post","text":"Друзья в социальных сетях – явление из 2010 года. Все, кто нужен, давно в мессенджерах. Поддерживать устаревающие списки друзей в публичных сетях бессмысленно и долго. Читать чьи-то новости – засорение мозга. <br><br>Чтобы расчистить место для нового, нужно не бояться избавляться от старого багажа – устаревающих идей, представлений, связей, мест работы. <br><br>Говорят, что лучшее – враг хорошего. Но верно и то, что хорошее – враг лучшего. Единственная преграда на пути к лучшему – это цепляние за то хорошее, что уже есть.","comments":{"count":8193},"likes":{"count":20872},"reposts":{"count":2065}},{"id":1692967,"from_id":1,"to_id":1,"date":1491331611,"post_type":"copy","text":"Magic💫 Процесс создания нового персонажа для официальных стикеров Telegram.","copy_post_date":1484496080,"copy_post_type":"post","copy_owner_id":768540,"copy_post_id":2810,"copy_text":"t.me\\/addstickers\\/lazypanda

Часть адреса https://api.vk.com/method/ можно запомнить. Все остальные методы просто добавляются к нему: https://api.vk.com/method/wall.getComments, https://api.vk.com/method/wall.getById и т.д.            

In [4]:
api_link = 'https://api.vk.com/method/'
method = 'wall.get'
response = requests.get(api_link + method + '?owner_id=1&count=2')

API VK отправляет ответ в формате json, который мы можем преобразовать в объекты питона с помощью модуля json.

In [5]:
import json
data = json.loads(response.text) 
print(type(data))

<class 'dict'>


Обычно в результате мы получаем словарь, в котором друг в друга вложены массивы, словари, снова массивы... Хотя там может быть довольная сложная структура данных, бояться не нужно: все подробно это описано в документации к API, но кроме того часто всё понятно и без дополнительных объяснений:

In [6]:
data

{'response': [237,
  {'comments': {'count': 8193},
   'date': 1492437980,
   'from_id': 1,
   'id': 1725537,
   'likes': {'count': 20872},
   'post_type': 'post',
   'reposts': {'count': 2065},
   'text': 'Друзья в социальных сетях – явление из 2010 года. Все, кто нужен, давно в мессенджерах. Поддерживать устаревающие списки друзей в публичных сетях бессмысленно и долго. Читать чьи-то новости – засорение мозга. <br><br>Чтобы расчистить место для нового, нужно не бояться избавляться от старого багажа – устаревающих идей, представлений, связей, мест работы. <br><br>Говорят, что лучшее – враг хорошего. Но верно и то, что хорошее – враг лучшего. Единственная преграда на пути к лучшему – это цепляние за то хорошее, что уже есть.',
   'to_id': 1},
  {'attachment': {'type': 'video',
    'video': {'access_key': '7a0ff2f6eca048e9b9',
     'date': 1484495980,
     'description': 'See this Instagram video by @brilekon',
     'duration': 0,
     'image': 'https://pp.userapi.com/c636924/u768540/vid

Например, попробуем извлечь только тексты постов:

In [7]:
print('Первый пост:\n', data["response"][1]["text"])
print('\nВторой пост:\n', data["response"][2]["text"])

Первый пост:
 Друзья в социальных сетях – явление из 2010 года. Все, кто нужен, давно в мессенджерах. Поддерживать устаревающие списки друзей в публичных сетях бессмысленно и долго. Читать чьи-то новости – засорение мозга. <br><br>Чтобы расчистить место для нового, нужно не бояться избавляться от старого багажа – устаревающих идей, представлений, связей, мест работы. <br><br>Говорят, что лучшее – враг хорошего. Но верно и то, что хорошее – враг лучшего. Единственная преграда на пути к лучшему – это цепляние за то хорошее, что уже есть.

Второй пост:
 Magic💫 Процесс создания нового персонажа для официальных стикеров Telegram.


Для удобства составление запроса и обращение к api можно записать в одну функцию:

In [8]:
def vk_api(method, **kwargs):
    api_request = 'https://api.vk.com/method/'+method + '?'
    api_request += '&'.join(['{}={}'.format(key, kwargs[key]) for key in kwargs])
    return json.loads(requests.get(api_request).text)

## Задания на семинар

### 1. Смотрим на другие параметры метода wall.get

Что ещё можно передать методу wall.get, кроме id пользователя и числа постов? Как это применить?

### 2. Скачиваем комментарии

Комментарии к постам скачиваются с помощью другого метода: https://vk.com/dev/wall.getComments

Ему нужно передавать идентификаторы записи, комментарии к которой мы хотим получить (эти идентификаторы нам поставляет метод wall.get). Обратите внимание, что VK API позволяет за одно обращение скачать не больше 100 записей и 100 комментариев. Дурова комментируют много, так что всё сразу достать не получится. Но если немного подумать, то можно сделать и это. Как?

Если ваша собственная стена открыта и записи там доступны без авторизации, можете скачать её, и посчитать, кто вас больше комментирует. Какие у комментаторов самые частотные слова?

### 3. Изучаем другие методы

Посмотрите список методов: https://vk.com/dev/methods. Некоторые методы доступны только после авторизации и получения токена доступа. 

Что еще можно скачать без авторизации? Изучите методы для работы с пользователями и группами, например, `users.get` и `group.members`.


## Домашнее задание

В домашнем задании вам нужно дописать ваш сайт:

1) Если у вас еще нет главной страницы, то нужно добавить главную страницу. На главной странице должно быть написано, кто автор сайта, где находится исходный код (ссылка на репозиторий), список приложений на вашем сайте. Пока что у всех должно быть одно приложение, которое разбирает майстемом текст и сообщает информацию о глаголах.

2) Нужно написать второе приложение, которое обращается к API VK. Вы можете выбрать любой из следующих вариантов:
    
 * Пользователь может ввести id двух открытых групп вконтакте, сайт выводит количество подписчиков каждой группы, а так же количество пользователей, которые подписаны на обе группы. Если введена закрытая группа, нужно сообщить об этом.
 * Пользователь вводит id группы, ваше приложение скачивает первые 1000 постов со стены группы (если постов меньше 1000, то скачивает все), и выводит на сайт 100 самых частотных словоформ. Если введена закрытая группа, нужно сообщить об этом.
 * Пользователь вводит id группы, ваше приложение смотрит на первые 1000 постов и распечатывает список 10 пользователей, которые написали больше всего комментариев. Если введена закрытая группа, нужно сообщить об этом.
 * Можно придумать свой вариант, но его нужно обсудить=)
 
 
Вот пример сайта: http://2017learnpython.pythonanywhere.com/